In [1]:
SEED = 42 
import numpy.random
import random
numpy.random.seed(SEED)
random.seed(SEED)
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import functools
from ema_workbench import (Model, RealParameter, Policy, Constant, Scenario, Constraint, 
                           ScalarOutcome, MultiprocessingEvaluator, ema_logging, perform_experiments)
from ema_workbench.util import ema_logging, save_results, load_results
from ema_workbench.em_framework.optimization import (EpsilonProgress, HyperVolume, ArchiveLogger)
from problem_formulation import get_model_for_problem_formulation
from ema_workbench.em_framework import sample_uncertainties
from ema_workbench.em_framework.evaluators import BaseEvaluator

%matplotlib inline  
# With this backend, the output of plotting commands is displayed inline within frontends like the Jupyter notebook, 
# directly below the code cell that produced it. The resulting plots will then also be stored in the notebook document.

ema_logging.log_to_stderr(ema_logging.INFO)
dike_model, planning_steps = get_model_for_problem_formulation(2) # assign problem_formulation_id

C:\Users\georg\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [2]:
'''
For different list of outcomes:
# 0 = 2-objective PF
# 1 = 3-objective PF
# 2 = 5-objective PF
# 3 = Disaggregate over locations
# 4 = Disaggregate over time
# 5 = Fully disaggregated
'''
ID_PF = 3    # assign problem_formulation_id
NR_STEPS = 2 # assign number of planning steps
dike_model, planning_steps = get_model_for_problem_formulation(ID_PF)

In [ ]:
%%time
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=10, policies=1000)

save_results(results, './results/open_exploration_policies_10000runs_2steps_ID_PF_3.tar.gz')

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 10 scenarios * 1000 policies * 1 model(s) = 10000 experiments


In [3]:
# results

In [ ]:
%%time
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=1000, policies=10) # using lhs sampling

save_results(results, './results/open_exploration_uncertainties_10000runs_2steps_ID_PF_3.tar.gz')

In [ ]:
experiments, outcomes = results

In [ ]:
# use the same 1,000 sampled scenarios, run for zero policy
sampled_scenarios = experiments.loc[:, [u.name for u in dike_model.uncertainties]]
sampled_scenarios.drop_duplicates(inplace=True)
ref_scenarios = [Scenario(i, **scenario.to_dict())
                 for i, scenario in pd.DataFrame.from_dict(sampled_scenarios).iterrows()] # sampled scenarios

# make a policy instance for zero policy
policy0 = {'DikeIncrease': 0, 'DaysToThreat': 0, 'RfR': 0}
ref_policy = {}
for key in dike_model.levers:
    _, s = key.name.split('_')
    if ' ' in s:
        s, _ = s.split(' ')
    ref_policy.update({key.name: policy0[s]})
ref_policy0 = Policy('baseline', **ref_policy) # policy0

In [ ]:
%%time
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=ref_scenarios, 
                                            policies=ref_policy0) # using lhs sampling

save_results(results, './results/open_exploration_uncertainties_policy0_2steps_ID_PF_3.tar.gz')